In [1]:
# import packages
import pandas as pd
import duckdb

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr",False)

In [29]:
# connect database
from pipelines.tasks.config.common import DUCKDB_FILE
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [31]:
# Liste des tables
db_tables = con.sql("SHOW TABLES").df() # Afficher le DataFrame 
print(db_tables)

               name
0      cog_communes
1      edc_communes
2  edc_prelevements
3     edc_resultats
4  laposte_communes


In [4]:
#view table edc_communes
edc_communes = con.execute("SELECT * FROM edc_communes LIMIT 2").fetchdf()
edc_communes

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date,de_dataset_datetime
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024,2025-02-28,20250203-075802
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024,2025-02-28,20250203-075802


In [6]:
#view table edc_prelevements
edc_prelevements = con.execute("SELECT * FROM edc_prelevements LIMIT 2").fetchdf()
edc_prelevements

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date,de_dataset_datetime
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qualité et non conforme aux références de qualité.,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,C,C,N,C,2024,2025-02-28,20250203-075802
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2024,2025-02-28,20250203-075802


In [7]:
#view table edc_resultats
edc_resultats = con.execute("SELECT * FROM edc_resultats LIMIT 2").fetchdf()
edc_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100138503,ALTMICR,1370,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,None,N,L,<10,µg/L,133,None,<=200 µg/L,0.0,7429-90-5,00100147233,2024,2025-02-28,20250203-075802
1,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100147232,2024,2025-02-28,20250203-075802


In [11]:
#table contenant unique prélèvement (referenceprel) avec nombre de résultat par prélèvement
query_upnbr = """
SELECT DISTINCT referenceprel, COUNT(cdparametre) AS nbresult
FROM edc_resultats
GROUP BY referenceprel
"""
nb_resultat_par_plv = con.sql(query_upnbr).fetchdf()
nb_resultat_par_plv

query_max_result = """
SELECT MAX(nbresult)
FROM nb_resultat_par_plv
"""
max_result = con.sql(query_max_result).fetchdf()
max_result


,max(nbresult)
0,669


In [14]:
query_upnbr = """
SELECT DISTINCT referenceprel, COUNT(cdparametre) AS nbresult
FROM edc_resultats
GROUP BY referenceprel
ORDER BY nbresult DESC
LIMIT 1
"""
nb_resultat_par_plv = con.sql(query_upnbr).fetchdf()
nb_resultat_par_plv

,referenceprel,nbresult
0,08300288944,669


In [15]:
query_08300288944 = """
SELECT *
FROM edc_resultats
WHERE referenceprel IN ('08300288944')
"""
table_08300288944 = con.sql(query_08300288944).fetchdf()
table_08300288944

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,083,08300288944,245T,1264,"2,4,5-T","2,4,5-T",None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,93-76-5,08300288238,2024,2025-02-28,20250203-075802
1,083,08300288944,24D,1141,"2,4-D","2,4-D",None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,94-75-7,08300288238,2024,2025-02-28,20250203-075802
2,083,08300288944,24DB,1142,"2,4-DB","2,4-DB",None,N,L,"<0,050",µg/L,133,"<=0,1 µg/L",None,0.0,94-82-6,08300288238,2024,2025-02-28,20250203-075802
3,083,08300288944,24DCP,1486,"DICHLOROPHÉNOL-2,4","Dichlorophénol-2,4",None,N,L,"<0,020",µg/L,133,None,None,0.0,120-83-2,08300288238,2024,2025-02-28,20250203-075802
4,083,08300288944,24DIE,2872,"2,4-D-ISOPROPYL ESTER","2,4-D-isopropyl ester",None,N,L,"<0,005",µg/L,133,"<=0,1 µg/L",None,0.0,94-11-1,08300288238,2024,2025-02-28,20250203-075802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,083,08300288944,VCLZ,1291,VINCHLOZOLINE,Vinchlozoline,None,N,L,"<0,005",µg/L,133,"<=0,1 µg/L",None,0.0,50471-44-8,08300288238,2024,2025-02-28,20250203-075802
669,083,08300288944,VMDT,1290,VAMIDOTHION,Vamidothion,None,N,L,"<0,005",µg/L,133,"<=0,1 µg/L",None,0.0,2275-23-2,08300288238,2024,2025-02-28,20250203-075802
670,083,08300288944,ZCYINE,7521,ZETACYPERMETHRINE,Zetacypermethrine,None,N,L,"<0,005",µg/L,133,"<=0,1 µg/L",None,0.0,97955-44-7,08300288238,2024,2025-02-28,20250203-075802
671,083,08300288944,ZIRA,1722,ZIRAME,Zirame,None,N,L,<100,µg/L,133,"<=0,1 µg/L",None,0.0,137-30-4,08300288238,2024,2025-02-28,20250203-075802


In [27]:
query_08300288944 = """
SELECT *
FROM edc_resultats
WHERE referenceprel IN ('08300288944')
"""
table_08300288944 = con.sql(query_08300288944).fetchdf()
table_08300288944

query_uentite_08300288944 = """
SELECT DISTINCT de_ingestion_date
FROM table_08300288944
"""
param_unique_08300288944 = con.sql(query_uentite_08300288944).fetchdf()
param_unique_08300288944

,de_ingestion_date
0,2025-02-28
